In [1]:
import gym
import numpy as np

In [2]:
env = gym.make('FrozenLake-v0')

In [3]:
S = env.observation_space.n
A = env.action_space.n

In [4]:
print("랜덤하게 움직여서 겜 한판 돌려보자!")

s = env.reset()
env.render()


while True:
    s, r, done, _ = env.step(env.action_space.sample())
    env.render()
    if done:
        break

랜덤하게 움직여서 겜 한판 돌려보자!

SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG


## constant

In [5]:
GAMMA = .9
EPISODES = 1000
THETA = 1e-4

## variables

In [6]:
# init policy, S x A x 1
pi = np.ones((S, A))
pi /= A
pi

array([[0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25]])

In [7]:
# init rewrads, S x A x 1
rewards = np.zeros((S, A))

In [8]:
# init transition matrix, A x S x S' x 1
Pss = np.zeros((A, S, S))

In [9]:
# init value function, S x 1
valuefunction = np.ones((S)) / S

In [10]:
# return list
return_list = []

## policy evaluation

Input $\pi$, the policy to be evaluated</br>
Algorithm parameter: a small threshold $\theta > 0$ determining accuracy of estimation</br>
Initialize $V(s)$, for all $s\in S^+$, arbitrarily except that $V (terminal) = 0$

Loop:</br>
&nbsp;&nbsp;&nbsp;$\Delta\leftarrow 0$</br>
&nbsp;&nbsp;&nbsp;Loop for each $s\in S$:</br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$v\leftarrow V(s)$</br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$V(s)\leftarrow \sum_a \pi (a|s) \sum_{s',r}p(s', r|s, a)[r+\gamma V(s')]$</br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\Delta \leftarrow \max (\Delta, |v-V(s)|)$</br>
until $\Delta < \theta$

In [11]:
action = np.arange(A)

def get_action(state):
    return np.random.choice(action, size=None, p=pi[state])

def get_transition(episodes):
    global rewards, return_list
    
    _rewards = np.zeros(rewards.shape)
    
    for _ in range(episodes):
        state = env.reset()
        while True:
            action = get_action(state)
            state_, _reward, done, _ = env.step(action)
            
            Pss[action][state][state_] += 1 # cumulate again and again
            _rewards[state][action] += _reward
            
            if done:
                break
            
            state = state_
                
    # average rewards
    _rewards /= episodes
    rewards = _rewards
    return_list.append(np.sum(rewards))

In [12]:
# POLICY EVALUATION

# 첨엔 100번
get_transition(100)

while True:
    delta = .0

    PssV = np.zeros((A, S))
    pss_sum = np.sum(Pss)
    for a in range(A):
        for s in range(S):
            _sum = 0.
            for s_ in range(S):
                _sum += Pss[a][s][s_]/pss_sum * valuefunction[s_]
            PssV[a][s] = _sum
            
    PssV *= GAMMA
    
    Rsum = np.zeros((S, A))
    for s in range(S):
        for a in range(A):
            Rsum[s][a] += rewards[s][a] + PssV[a][s]
    
    newV = np.zeros((S))
    for s in range(S):
        _sum = 0.
        for a in range(A):
            newV[s] += pi[s][a] * Rsum[s][a]
            
    delta = max(delta, np.sum(np.abs(valuefunction - newV)))
    
    if delta < THETA:
        break
        
    valuefunction = np.array(newV)
    
    # 그 담번엔 20번씩
    get_transition(EPISODES)

In [13]:
valuefunction

array([8.91149035e-15, 1.28104729e-13, 2.22833996e-11, 2.90979244e-14,
       1.64030633e-13, 0.00000000e+00, 2.27063363e-09, 0.00000000e+00,
       2.92895437e-11, 3.66863060e-09, 4.72783949e-06, 0.00000000e+00,
       0.00000000e+00, 3.66060827e-06, 2.25294061e-03, 0.00000000e+00])

In [14]:
rewards

array([[0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.003, 0.003, 0.003],
       [0.   , 0.   , 0.   , 0.   ]])

## policy improvement

$policy-stable \leftarrow true$</br>
For each $s\in S$:</br>
&nbsp;&nbsp;&nbsp;&nbsp;$old-action \leftarrow \pi(s)$</br>
&nbsp;&nbsp;&nbsp;&nbsp;$\pi(s) \leftarrow \arg\max_a \sum_{s',r} p(s', r | s, a)[r + \gamma V(s')]$</br>
&nbsp;&nbsp;&nbsp;&nbsp;If $old-action \neq \pi(s)$, then $policy-stable\leftarrow false$</br>
If $policy-stable$, then stop and return $V\approx v_*$ and $]pi \approx \pi_*$

In [15]:
# POLICY IMPROVEMENT


old_action = np.array(pi)

PssV = np.zeros((A, S))
pss_sum = np.sum(Pss)
for a in range(A):
    for s in range(S):
        _sum = 0.
        for s_ in range(S):
            _sum += Pss[a][s][s_]/pss_sum * valuefunction[s_]
        PssV[a][s] = _sum

PssV *= GAMMA

new_action = np.zeros((S, A))
for s in range(S):
    for a in range(A):
        Rsum[s][a] += rewards[s][a] + PssV[a][s]
for s in range(S):
    if np.min(Rsum[s]) != np.max(Rsum[s]):
        new_action[s][np.argmax(Rsum[s])] = 1.
    else:
        new_action[s] = pi[s]

In [16]:
pi

array([[0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25]])

## policy iteration

In [17]:
# 첨엔 100번
get_transition(100)

while True:

    '''
    POLICY EVALUATION
    '''

    while True:
        delta = .0

        PssV = np.zeros((A, S))
        pss_sum = np.sum(Pss)
        for a in range(A):
            for s in range(S):
                _sum = 0.
                for s_ in range(S):
                    _sum += Pss[a][s][s_]/pss_sum * valuefunction[s_]
                PssV[a][s] = _sum

        PssV *= GAMMA

        Rsum = np.zeros((S, A))
        for s in range(S):
            for a in range(A):
                Rsum[s][a] += rewards[s][a] + PssV[a][s]

        newV = np.zeros((S))
        for s in range(S):
            _sum = 0.
            for a in range(A):
                newV[s] += pi[s][a] * Rsum[s][a]

        delta = max(delta, np.sum(np.abs(valuefunction - newV)))

        if delta < THETA:
            break

        valuefunction = np.array(newV)

        get_transition(EPISODES)
        
        return_list.append(np.sum(rewards))



    """
    POLICY IMPROVEMENT
    """

    policy_stable = True

    old_action = np.array(pi)

    PssV = np.zeros((A, S))
    pss_sum = np.sum(Pss)
    for a in range(A):
        for s in range(S):
            _sum = 0.
            for s_ in range(S):
                _sum += Pss[a][s][s_]/pss_sum * valuefunction[s_]
            PssV[a][s] = _sum

    PssV *= GAMMA

    new_action = np.zeros((S, A))
    for s in range(S):
        for a in range(A):
            Rsum[s][a] += rewards[s][a] + PssV[a][s]
    for s in range(S):
        if np.min(Rsum[s]) != np.max(Rsum[s]):
            new_action[s][np.argmax(Rsum[s])] = 1.
        else:
            new_action[s] = pi[s]


    if not (old_action == new_action).all():
        policy_stable = False
        pi = np.array(new_action)
        
    if policy_stable:
        print("We have pi* and v*!")
        print("pi*")
        print(pi)
        print("v*")
        print(valuefunction)
        break

We have pi* and v*!
pi*
[[0.   0.   1.   0.  ]
 [0.   0.   0.   1.  ]
 [1.   0.   0.   0.  ]
 [0.   1.   0.   0.  ]
 [0.   1.   0.   0.  ]
 [0.25 0.25 0.25 0.25]
 [0.   0.   1.   0.  ]
 [0.25 0.25 0.25 0.25]
 [0.   1.   0.   0.  ]
 [0.   1.   0.   0.  ]
 [0.   0.   1.   0.  ]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.   0.   0.   1.  ]
 [0.   0.   1.   0.  ]
 [0.25 0.25 0.25 0.25]]
v*
[2.21057395e-10 2.98031834e-09 9.13250289e-08 1.17547844e-10
 1.16523952e-09 0.00000000e+00 3.24644594e-06 0.00000000e+00
 4.94036832e-08 2.61740804e-06 4.22549120e-04 0.00000000e+00
 0.00000000e+00 1.41873138e-04 6.62734924e-02 0.00000000e+00]


In [18]:
# for a in range(A):
#     for s in range(S):
#         for s_ in range(S):
#             print(f"(a:{a}, s:{s}) -> s':{s_} : {Pss[a][s][s_]}")

In [19]:
s = env.reset()
cnt_success = 0
trial = 1000

for _ in range(trial):
    while True:
        s, r, done, _ = env.step(get_action(s))

        cnt_success += r

        if done:
            s = env.reset()
            break
        
print(f"policy iteration 으로 풀었을 때, 에이전트 퍼포먼스: {trial}회 중 {cnt_success:n}회 성공 -> {cnt_success/trial * 100:.2f}%")

policy iteration 으로 풀었을 때, 에이전트 퍼포먼스: 1000회 중 67회 성공 -> 6.70%


In [20]:
s = env.reset()
cnt_success = 0
trial = 1000

for _ in range(trial):
    while True:
        s, r, done, _ = env.step(env.action_space.sample())

        cnt_success += r

        if done:
            s = env.reset()
            break
        
print(f"아무곳이나 가는 에이전트 퍼포먼스: {trial}회 중 {cnt_success:n}회 성공 -> {cnt_success/trial * 100:.2f}%")

아무곳이나 가는 에이전트 퍼포먼스: 1000회 중 12회 성공 -> 1.20%
